## LinearRegression temp

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#데이터 불러오기 제외된 코드
X = wine.drop(["type","quality"], axis = 1)
y = wine.quality

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1)

model = LinearRegression(fit_intercept = True, n_jobs=None, normalize = True)
model.fit(X_train, y_train)#모델 훈련 코드
print("coef = ", model.coef_) #기울기 / 없어도 상관없다.
print("-"*30) #보기 편하게 넣음
print("intercept = ", model.intercept_) # 절편

y_pred = model.predict(X_test)

np.round(np.sqrt(mean_squared_error(y_test, y_pred)),2)

# 규제가 있는 선형모델

## 1. 릿지

In [ ]:
from sklearn.linear_model import Ridge

model_name = "ridge"
alpha = 0.05
fig = plt.figure(figsize=(6,3))
ax = fig.add_subplot(111)
ridge = Ridge(alpha=alpha)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
rmse = np.round(np.sqrt(mean_squared_error(y_test, y_pred)),3)
coef = pd.Series(data=ridge.coef_, index=X_train.columns).sort_values()
ax.bar(coef.index, coef.values)
ax.set_xticklabels(coef.index, rotation = 90)
ax.set_title("{}: alpha = {}, rmse = {}".format(model_name, alpha, rmse))

## 2. 라소

In [ ]:
from sklearn.linear_model import Lasso

model_name = "lasso"
alpha = 0.05
fig = plt.figure(figsize=(6,3))
ax = fig.add_subplot(111)
lasso = Lasso(alpha=alpha)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
rmse = np.round(np.sqrt(mean_squared_error(y_test, y_pred)),3)
coef = pd.Series(data=lasso.coef_, index=X_train.columns).sort_values()
ax.bar(coef.index, coef.values)
ax.set_xticklabels(coef.index, rotation = 90)
ax.set_title("{}: alpha = {}, rmse = {}".format(model_name, alpha, rmse))

## K-최근접 이웃

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV

#데이터 읽어오기
breast_cancer = pd.read_csv("breast-cancer-wisconsin.data", header =None)

#컬럼명 변경
breast_cancer.columns = ["id_number", "cluster_thickness",
                         "unif_cell_size", "unif_cell_shape",
                         "marg_adhesion", "single_epith_cell_size",
                         "bare_nuclei", "bland_chromatin",
                         "normal_nucleoli", "mitoses", "class"]

# ?을 NA로 바꾼 후 NA를 최빈값으로 만들기
breast_cancer["bare_nuclei"]= breast_cancer["bare_nuclei"].replace("?",np.NAN)
breast_cancer["bare_nuclei"] = breast_cancer["bare_nuclei"]= breast_cancer["bare_nuclei"].\
                                                             fillna(breast_cancer["bare_nuclei"].value_counts().index[0])

#class 0,1로 변환
breast_cancer["cancer_ind"] = 0
breast_cancer.loc[breast_cancer["class"]==4, "cancer_ind"] = 1

# X,y 만들기
X = breast_cancer.drop(["id_number","class","cancer_ind"], axis=1)
y = breast_cancer.cancer_ind

#훈련데이터, 테스터데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

#데이터 표준화 작업 객체생성
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train) #훈련데이터,테스트 테이터를 표준화
X_test_scaled = scaler.transform(X_test)

print("-"*20)
print("accuracy =",accuracy_score(y_test, y_pred))
print("roc_auc = ",roc_auc_score(y_test, y_pred))

knn = KNeighborsClassifier(n_neighbors=3) # 기존데이터를 선택하여 거리를 
                                          # 비교한 후 분류
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)
accuracy_score(y_test, y_pred)
roc_auc_score(y_test, y_pred)

# 그리드 서치를 통해 하이퍼파라미터의 최적값 찾기
grid_search = GridSearchCV(knn, {"n_neighbors": [1,2,3,4,5]},\
                          n_jobs=-1, cv = 7, scoring = "roc_auc")
grid_search.fit(X_train_scaled, y_train)

knn_best = grid_search.best_estimator_
y_pred = knn_best.predict(X_test_scaled)
accuracy_score(y_test, y_pred)
roc_auc_score(y_test, y_pred)

print("-"*20)
print("accuracy2 =",accuracy_score(y_test, y_pred))
print("roc_auc2 = ",roc_auc_score(y_test, y_pred))